<a href="https://colab.research.google.com/github/Rai-shwith/learn-langgraph/blob/main/LangGraphLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lang Graph Learnings:
 This will be my notebook to learn and practice Langgraph \\
 source: [freeCodeCamp](https://www.youtube.com/watch?v=jGg_1h0qzaM&t=4209s)

## Installation

In [1]:
!pip install langgraph --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.7 MB/s eta 0:00:00


## Imports

In [ ]:
from typing import Dict,TypedDict
from langgraph.graph import StateGraph

## State of the Langgraph

In [ ]:
class AgentState(TypedDict):
  message:str

## Node of the Graph

In [ ]:
def greeting_node(state:AgentState)->AgentState:
  """Simple Node that greets the users"""
  state["message"]= f'Hey, {state["message"]} How are you doing?'
  return state

## Graph initialization

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("Greeter",greeting_node)
graph.set_entry_point("Greeter")
graph.set_finish_point("Greeter")
app = graph.compile()

## Working with compiled app

In [ ]:
result = app.invoke({"message":"Alex"})
result["message"]

'Hey, Alex How are you doing?'

## Exercise: Personalized Complement Agent

In [ ]:
class TaskState(TypedDict):
  name:str

def compliment(state:TaskState)->TaskState:
  """Creates a Personalized Compliment"""
  state["name"] = f"{state['name']}, you're doing an amazing job!"
  return state

graph = StateGraph(TaskState)
graph.add_node('compliment',compliment)
graph.set_entry_point('compliment')
graph.set_finish_point('compliment')
app = graph.compile()
result = app.invoke({'name':'Bob'})
result["name"]


"Bob, you're doing an amazing job!"

## Multiple Inputs

In [ ]:
from typing import List, TypedDict
from langgraph.graph import StateGraph

In [ ]:
class AgentState(TypedDict):
  values:list[int]
  name: str
  result: str

In [ ]:
def process_values(state:AgentState)->AgentState:
  """This function handles multiple inputs"""
  state['result'] = f"Hello bro, Is your name {state['name']}, cuz your sum is {state['values']}"
  return state

In [ ]:
graph  = StateGraph(AgentState)
graph.add_node("processor",process_values)
graph.set_entry_point("processor")
graph.set_finish_point("processor")
app = graph.compile()

In [ ]:
result = app.invoke({"values":[1,2,3,4],"name":"Ash"})
result

{'values': [1, 2, 3, 4],
 'name': 'Ash',
 'result': 'Hello bro, Is your name Ash, cuz your sum is [1, 2, 3, 4]'}

## Sequential Graph

In [ ]:
from langgraph.graph import StateGraph
from typing import TypedDict

In [ ]:
class AgentState(TypedDict):
  name:str
  age:int
  final:str

def first_node(state:AgentState)->AgentState:
  """This is the first Node"""
  state['final'] = f"Hello, {state['name']}. "
  return state


def second_node(state:AgentState)->AgentState:
  """This is the second Node"""
  state['final'] = state['final']+ f"You are {state['age']} years old"
  return state


graph = StateGraph(AgentState)
graph.add_node("first",first_node)
graph.add_node("second",second_node)
graph.set_entry_point("first")
graph.set_finish_point("second")
graph.add_edge('first','second')
app = graph.compile()
res = app.invoke({"name":"Alex","age":20})
res

{'name': 'Alex', 'age': 20, 'final': 'Hello, Alex. You are 20 years old'}

## Exercise for Sequential Graph

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
  name:str
  age:int
  skills:list[str]
  message:str

def greet(state:AgentState)-> AgentState:
  """This node greets"""
  state['message'] = f"{state['name']}, welcome to the system!"
  return state

def age_node(state:AgentState)->AgentState:
  """This adds age details to the message"""
  state['age'] = f"{state['message']} You are {state['age']} years old!"

def skill_adder(state:AgentState)->AgentState:
  """This section adds the skill details"""
  state['message'] = state['message'] + f" You have skills in: {', '.join(state['skills'][:-1])+', and '+ state['skills'][-1] if len(state['skills']) > 1 else state['skills'][0]}"
  return state

graph = StateGraph(AgentState)
graph.add_node("greet",greet)
graph.add_node("age",age_node)
graph.add_node('skill',skill_adder)
graph.set_entry_point("greet")
graph.set_finish_point('skill')
graph.add_edge("greet","age")
graph.add_edge("age","skill")
app = graph.compile()
app.invoke({'name':"Linda","age":31,"skills":["Python","Machine Learning", "LangGraph"]})

{'name': 'Linda',
 'age': 31,
 'skills': ['Python', 'Machine Learning', 'LangGraph'],
 'message': 'Linda, welcome to the system! You have skills in: Python, Machine Learning, and LangGraph'}

## Conditional Agent

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START,END

class AgentState(TypedDict):
  num1:int
  num2:int
  op:str
  res:int

def adder(state:AgentState)->AgentState:
  """This node adds"""
  state['res'] = state['num1'] + state['num2']
  return state

def subtracter(state:AgentState)->AgentState:
  """This node subtracts"""
  state['res'] = state['num1'] - state['num2']
  return state

def decide_node(state:AgentState)->str:
  """This is the router function"""
  if state['op']=="+":
    return "addition_edge"
  elif state['op']=="-":
    return "subtraction_edge"

graph=StateGraph(AgentState)
graph.add_node("add",adder)
graph.add_node("sub",subtracter)
graph.add_node("router",lambda state:state)
graph.add_edge(START,"router")
graph.add_conditional_edges("router",decide_node,{
    "addition_edge":"add",
    "subtraction_edge":"sub"
})
graph.add_edge("add",END)
graph.add_edge("sub",END)
app = graph.compile()
app.invoke({"num1":20,"num2":10,"op":"-"})

{'num1': 20, 'num2': 10, 'op': '-', 'res': 10}

## Exercise: 2 Conditional Edges

In [11]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict

class AgentState(TypedDict):
  num1:int
  num2:int
  num3:int
  num4:int
  op1:str
  op2:str
  res1:int
  res2:int

def decide_first(state:AgentState)->str:
  """Decides which edge to choose in first """
  if state["op1"] == "+":
    return "first_add"
  else:
    return "first_sub"

def decide_sec(state:AgentState)->str:
  """Decides which edge to choose in second """

  if state["op2"] == "+":
    return "sec_add"
  else:
    return "sec_sub"

def adder1(state:AgentState)->AgentState:
  """Adds 2 numbers and stores the result1"""
  state["res1"] = state["num1"]+state["num2"]
  return state

def adder2(state:AgentState)->AgentState:
  """Adds 2 numbers and stores the result2"""
  state["res2"] = state["num3"]+state["num4"]
  return state

def sub1(state:AgentState)->AgentState:
  """subtracts 2 numbers and stores the result1"""
  state["res1"] = state["num1"]-state["num2"]
  return state

def sub2(state:AgentState)->AgentState:
  """subtracts 2 numbers and stores the result2"""
  state["res2"] = state["num3"]-state["num4"]
  return state

def show(state:AgentState)->AgentState:
  """This node is used to show the result"""
  print(f"{state['num1']} {state['op1']} {state['num2']} = {state['res1']}")
  print(f"{state['num3']} {state['op2']} {state['num4']} = {state['res2']}")
  return state


graph = StateGraph(AgentState)
graph.add_node("add1",adder1)
graph.add_node("sub1",sub1)
graph.add_node("add2",adder2)
graph.add_node("sub2",sub2)
graph.add_node("router1",lambda state:state)
graph.add_node("router2",lambda state:state)
graph.add_node("show",show)
graph.add_conditional_edges("router1",decide_first,{
    "first_add":"add1",
    "first_sub":"sub1"
})
graph.add_conditional_edges("router2",decide_sec,{
    "sec_add":"add2",
    "sec_sub":"sub2"
})
graph.add_edge(START,"router1")
graph.add_edge("add1","router2")
graph.add_edge("sub1","router2")
graph.add_edge("add2","show")
graph.add_edge("sub2","show")
graph.add_edge("show",END)
app = graph.compile()
app.invoke({"num1":10,"num2":5,"num3":30,"num4":20,"op1":"-","op2":"-"})

10 - 5 = 5
30 - 20 = 10


{'num1': 10,
 'num2': 5,
 'num3': 30,
 'num4': 20,
 'op1': '-',
 'op2': '-',
 'res1': 5,
 'res2': 10}

## Loops

In [18]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
import random
import time

class AgentState(TypedDict):
  name:str
  nums:list[int]
  count:int


def greet(state:AgentState)->AgentState:
  """Greets user"""
  state["name"] = f"Hi there, {state['name']}"
  state["count"] = 0
  state['nums'] = []
  return state

def rnd(state:AgentState)->AgentState:
  """Appends a random no. to list"""
  state["nums"].append(random.randint(0,10))
  state["count"]+=1
  return state

def looper(state:AgentState)->str:
  """Decides to loop or not"""
  if state["count"] < 5:
    print("looping...")
    # time.sleep(0.5)
    return "loop"
  print("Comming out!")
  return "exit"

graph = StateGraph(AgentState)
graph.add_node("greet",greet)
graph.add_node("random",rnd)
graph.add_edge(START,"greet")
graph.add_edge("greet","random")
graph.add_conditional_edges("random",looper,{
    "loop":"random",
    "exit":END
})
app = graph.compile()
app.invoke({"name":"Ashwith"})

looping...
looping...
looping...
looping...
Comming out!


{'name': 'Hi there, Ashwith', 'nums': [10, 4, 3, 3, 5], 'count': 5}

## Exercise: Automatic Higher or Lower Game

In [38]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict
import random

class AgentState(TypedDict):
  name:str
  lo:int
  hi:int
  guesses:list[int]
  count:int
  correct:bool


def setup(state:AgentState)->AgentState:
  """initailizes the params"""
  state['name'] = input("Enter your name: ")
  print("Hello ",state['name']," welcome to number guesser \n think of a number.")
  state['lo'] = int(input("Enter the lower limit: "))
  state['hi'] = int(input("Enter the higher limit: "))
  state["count"] = 14
  state["guesses"]=[]
  state["correct"] = False
  return state

def guess(state:AgentState)->AgentState:
  """Guesses a number based on limits"""
  # guess = random.randint(state['lo'],state['hi']+1)
  guess = (state['lo']+state['hi'])//2

  state["guesses"].append(guess)
  state["count"]-=1
  return state

def check(state:AgentState)->AgentState:
  """Checks if the number is correct or not"""
  state['correct'] = input(f"Is your guess {state['guesses'][-1]}:(y/n): ") == 'y'
  return state

def decide(state:AgentState)->str:
  """Decides to exit or ask for hint"""
  if not state['correct'] and state["count"] > 0:
    return "ask_hints"
  if state['correct'] and state["count"] != 0:
    print("Woohooo I guessed it right!!!")
  else:
    print("Sorry i couldn't guess")
  return "exit"


def hint(state:AgentState)->AgentState:
  """Asks for the hints"""
  hint = input("Enter (h) if the guess was higher else (l): ")
  if hint == 'h':
    state['hi']=state['guesses'][-1]-1
    return state
  elif hint == 'l':
    state['lo']=state['guesses'][-1]+1
    return state
  else:
    print("you typed something other than (h/l)")
    return hint(state)h

graph = StateGraph(AgentState)
graph.add_node('set',setup)
graph.add_node('guess',guess)
graph.add_node('hint',hint)
graph.add_node('check',check)
graph.add_edge(START,'set')
graph.add_edge("set","guess")
graph.add_edge("guess","check")
graph.add_conditional_edges("check",decide,{
    "ask_hints":"hint",
    "exit":END
})
graph.add_edge("hint","guess")
app = graph.compile()
app.invoke({})

Enter your name: anvi
Hello  anvi  welcome to number guesser 
 think of a number.
Enter the lower limit: 0
Enter the higher limit: 1000
Is your guess 500:(y/n): n
Enter (h) if the guess was higher else (l): l
Is your guess 750:(y/n): n
Enter (h) if the guess was higher else (l): h
Is your guess 625:(y/n): n
Enter (h) if the guess was higher else (l): h
Is your guess 562:(y/n): n
Enter (h) if the guess was higher else (l): h
Is your guess 531:(y/n): n
Enter (h) if the guess was higher else (l): l
Is your guess 546:(y/n): n
Enter (h) if the guess was higher else (l): h
Is your guess 538:(y/n): n
Enter (h) if the guess was higher else (l): l
Is your guess 542:(y/n): n
Enter (h) if the guess was higher else (l): l


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT